In [45]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import datetime
import os 
import json
import requests
import gmaps

#Use gkey for google apis
gkey = "" 
gmaps.configure(api_key=gkey)

In [46]:
#read the real estate data file of CA for 2012 to 2019 from Redfin
#creating data frame from the file
ca_data_df = pd.read_csv("../Resources/ca_data.csv")

In [47]:
#cleaning the data, creating a new data frame for required columns 
ca_clean_df = ca_data_df[["State","Region","City","Median Sale Price","Period Begin"]]
ca_clean_df["City"].fillna("NA", inplace=True)

#converting the "Median Sale Price" column to int
ca_clean_df["Median Sale Price"] = ca_clean_df["Median Sale Price"].str.replace('$', '')
ca_clean_df["Median Sale Price"] = ca_clean_df["Median Sale Price"].str.replace('K', '')
ca_clean_df["Median Sale Price"] = ca_clean_df["Median Sale Price"].str.replace(",",'')
ca_clean_df["Median Sale Price"] = ca_clean_df["Median Sale Price"].astype(float)
ca_clean_df["Median Sale Price"] = ca_clean_df["Median Sale Price"]*1000

#converting the date value to year
ca_clean_df['Month'] = pd.DatetimeIndex(ca_clean_df['Period Begin']).month
ca_clean_df['Year'] = pd.DatetimeIndex(ca_clean_df['Period Begin']).year

#Renaming the column names
ca_clean_df = ca_clean_df.rename(columns={'Region': 'County', 'Period Begin': 'Date'})


C:\Users\bbcho\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\bbcho\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\bbcho\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [48]:
ca_clean_df.head()

,State,County,City,Median Sale Price,Date,Month,Year
0,California,"East Hemet, CA",East Hemet,88000.0,2/1/2012,2,2012
1,California,"Temescal Valley, CA",Temescal Valley,283000.0,5/1/2012,5,2012
2,California,"Monterey Park, CA",Monterey Park,500000.0,7/1/2013,7,2013
3,California,"La Quinta, CA",La Quinta,460000.0,1/1/2014,1,2014
4,California,"Moreno Valley, CA",Moreno Valley,228000.0,4/1/2014,4,2014


In [49]:
#creating data frame for cities to process
city_names = ["Fremont", "San Jose", "Santa Clara", "Oakland", "Hayward"]
ca_city_df = ca_clean_df[ca_clean_df.City.isin(city_names)]
#creating csv file for this df
ca_city_df.to_csv("../output/5_city_data.csv")

In [51]:
ca_city_df.head()

,State,County,City,Median Sale Price,Date,Month,Year
50,California,"Santa Clara, CA",Santa Clara,465000.0,2/1/2012,2,2012
64,California,"Santa Clara, CA",Santa Clara,690000.0,10/1/2013,10,2013
80,California,"Oakland, CA",Oakland,610000.0,5/1/2015,5,2015
109,California,"Santa Clara, CA",Santa Clara,541000.0,3/1/2012,3,2012
193,California,"Oakland, CA",Oakland,499000.0,6/1/2014,6,2014


In [52]:
#creating data frame for counties to process
county_names = ["San Francisco, CA", "Alameda, CA", "San Mateo, CA", "Napa, CA", "Santa Clara, CA"]
ca_county_df = ca_clean_df[ca_clean_df.County.isin(county_names)]
#creating csv file for this df
ca_county_df.to_csv("../output/5_county_data.csv")

In [53]:
ca_county_df.head()

,State,County,City,Median Sale Price,Date,Month,Year
50,California,"Santa Clara, CA",Santa Clara,465000.0,2/1/2012,2,2012
64,California,"Santa Clara, CA",Santa Clara,690000.0,10/1/2013,10,2013
84,California,"Alameda, CA",Alameda,782000.0,9/1/2015,9,2015
109,California,"Santa Clara, CA",Santa Clara,541000.0,3/1/2012,3,2012
125,California,"Alameda, CA",Alameda,488000.0,9/1/2012,9,2012


In [54]:
#Getting lat lng coordinates of Cities
for index,row in ca_city_df.iterrows():
    target_city = row["City"]
    # Build the endpoint URL
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
                  'address={0}&key={1}').format(target_city, gkey)
    geo_data = requests.get(target_url).json()
    ca_city_df.at[index, "lat"] = geo_data["results"][0]["geometry"]["location"]["lat"]
    ca_city_df.at[index, "lng"] = geo_data["results"][0]["geometry"]["location"]["lng"]

C:\Users\bbcho\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\bbcho\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


NameError: name 'ca_house_df' is not defined

In [56]:
#saving data to file
ca_city_df.to_csv("../output/ca_city_gmap_input.csv")
ca_city_df.head()

,State,County,City,Median Sale Price,Date,Month,Year,lat,lng
50,California,"Santa Clara, CA",Santa Clara,465000.0,2/1/2012,2,2012,37.354108,-121.955236
64,California,"Santa Clara, CA",Santa Clara,690000.0,10/1/2013,10,2013,37.354108,-121.955236
80,California,"Oakland, CA",Oakland,610000.0,5/1/2015,5,2015,37.804364,-122.271114
109,California,"Santa Clara, CA",Santa Clara,541000.0,3/1/2012,3,2012,37.354108,-121.955236
193,California,"Oakland, CA",Oakland,499000.0,6/1/2014,6,2014,37.804364,-122.271114


In [57]:
#Getting lat lng coordinates of Cities
for index,row in ca_county_df.iterrows():
    target_county = row["County"]
    # Build the endpoint URL
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
                  'address={0}&key={1}').format(target_county, gkey)
    geo_data = requests.get(target_url).json()
    ca_county_df.at[index, "lat"] = geo_data["results"][0]["geometry"]["location"]["lat"]
    ca_county_df.at[index, "lng"] = geo_data["results"][0]["geometry"]["location"]["lng"]

In [58]:
#saving data to file
ca_county_df.to_csv("../output/ca_county_gmap_input.csv")
ca_county_df.head()

,State,County,City,Median Sale Price,Date,Month,Year,lat,lng
50,California,"Santa Clara, CA",Santa Clara,465000.0,2/1/2012,2,2012,37.354108,-121.955236
64,California,"Santa Clara, CA",Santa Clara,690000.0,10/1/2013,10,2013,37.354108,-121.955236
84,California,"Alameda, CA",Alameda,782000.0,9/1/2015,9,2015,37.765206,-122.241636
109,California,"Santa Clara, CA",Santa Clara,541000.0,3/1/2012,3,2012,37.354108,-121.955236
125,California,"Alameda, CA",Alameda,488000.0,9/1/2012,9,2012,37.765206,-122.241636
